## Imports & Parameters

In [75]:
import requests
from bs4 import BeautifulSoup
import json
from html import unescape
from datetime import date

In [ ]:
base_url = "https://www.boletinoficial.gob.ar"
fecha_desde = date.today().strftime("%d/%m/%Y")
fecha_hasta = date.today().strftime("%d/%m/%Y")
text = 'energía'

## Get

In [ ]:
url_busqueda = "/busquedaAvanzada/realizarBusqueda"

payload = {
    "params": f'''{{
        "busquedaRubro":false,
        "hayMasResultadosBusqueda":true,
        "ejecutandoLlamadaAsincronicaBusqueda":false,
        "ultimaSeccion":"",
        "filtroPorRubrosSeccion":false,
        "filtroPorRubroBusqueda":false,
        "filtroPorSeccionBusqueda":false,
        "busquedaOriginal":true,
        "ordenamientoSegunda":false,
        "seccionesOriginales":[1],
        "ultimoItemExterno":null,
        "ultimoItemInterno":null,
        "texto":"{text}",
        "rubros":[],
        "nroNorma":"",
        "anioNorma":"",
        "denominacion":"",
        "tipoContratacion":"",
        "anioContratacion":"",
        "nroContratacion":"",
        "fechaDesde":"{fecha_desde}",
        "fechaHasta":"{fecha_hasta}",
        "todasLasPalabras":true,
        "comienzaDenominacion":true,
        "seccion":[1],
        "tipoBusqueda":"Avanzada",
        "numeroPagina":1,
        "ultimoRubro":""
    }}''',
    "array_volver": "[]"
}

headers = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": base_url,
    "Referer": "https://www.boletinoficial.gob.ar/busquedaAvanzada/all",
    "User-Agent": "Mozilla/5.0"
}

response = requests.post(base_url + url_busqueda, data=payload, headers=headers)
print(response.status_code)


200


## Parse response

In [ ]:
data = json.loads(response.text)
html = data["content"]["html"]

html_limpio = unescape(html)
soup = BeautifulSoup(html_limpio, 'html.parser')

avisos = []
for a in soup.select("a[href]"):
    aviso = a.find("div", class_="linea-aviso")
    if aviso:
        link = a["href"]
        ministerio = aviso.select_one(".item").get_text(strip=True)
        resolucion = aviso.select_one(".item-detalle small").get_text(strip=True)
        fecha_pub = aviso.select(".item-detalle small")[1].get_text(strip=True)
        detalles = aviso.find_all("p", class_="item-detalle")
        referencia_contenido = detalles[2] if len(detalles) > 2 else None
        
        referencia = ""
        contenido = ""
        if referencia_contenido:
            small = referencia_contenido.find("small")
            if small:
                partes = list(small.stripped_strings)
                if len(partes) >= 1:
                    referencia = partes[0]
                if len(partes) > 1:
                    contenido = " ".join(partes[1:])

        avisos.append({
            "resolucion": resolucion,
            "ministerio": ministerio,
            "link": link,
            "fecha_publicacion": fecha_pub,
            "referencia": referencia,
            "contenido":contenido,
            "detalle": None 
        })

## Get content

In [81]:
for aviso in avisos:
    print(aviso["resolucion"])
    print(aviso["ministerio"])
    print(aviso["link"])
    print(aviso["fecha_publicacion"])
    print(aviso["referencia"])
    print(aviso["contenido"])
    print("--------")


Resolución 689/2025
MINISTERIO DE ECONOMÍA
/detalleAviso/primera/325995/20250527?busqueda=1
Fecha de Publicacion: 27/05/2025
RESOL-2025-689-APN-MEC
Subsecretaría de Transición y Planeamiento Energético de la Secretaría de Energía del Ministerio de Economía.Que la presente designación transitoria queda...
--------
Resolución 685/2025
MINISTERIO DE ECONOMÍA
/detalleAviso/primera/325994/20250527?busqueda=1
Fecha de Publicacion: 27/05/2025
RESOL-2025-685-APN-MEC
Subsecretaría de Transición y Planeamiento Energético de la Secretaría de Energía del Ministerio de Economía.Que la presente designación transitoria queda...
--------
